In [105]:
import pandas as pd


In [106]:
df = pd.read_csv('C:\\Users\\prani\\OneDrive\\Desktop\\Occupancy_Estimation.csv')

In [107]:
import numpy as np
import pandas as pd
import plotly.graph_objs as go
from plotly.offline import plot

# Assuming 'df' is already defined and contains the necessary data

# Handle special index types
if isinstance(df, (pd.DatetimeIndex, pd.MultiIndex)):
    df = df.to_frame(index=False)

# Remove any pre-existing indices for ease of use in the code
df = df.reset_index().drop('index', axis=1, errors='ignore')
df.columns = [str(c) for c in df.columns]  # Update columns to strings in case they are numbers

# Get numeric columns
numeric_columns = df.select_dtypes(include='number').columns.tolist()

# Initialize a list to store all figures
all_figures = []

# Iterate over each numeric column
for column in numeric_columns:
    # Select data and compute histogram
    s = df[~pd.isnull(df[column])][[column]]
    chart, labels = np.histogram(s[column], bins=20)

    # Compute KDE
    import scipy.stats as sts
    kde = sts.gaussian_kde(s[column])
    kde_data = kde.pdf(np.linspace(labels.min(), labels.max()))

    # Create Plotly traces
    charts = [
        go.Bar(x=labels[1:], y=chart, name='Histogram'),
        go.Scatter(
            x=list(range(len(kde_data))), y=kde_data, name='KDE',
            yaxis='y2', xaxis='x2',
            line={'shape': 'spline', 'smoothing': 0.3}, mode='lines'
        )
    ]

    # Create Plotly figure
    figure = go.Figure(data=charts, layout=go.Layout({
        'barmode': 'group',
        'legend': {'orientation': 'h'},
        'title': {'text': f'{column} Histogram (bins: 20) w/ KDE'},
        'xaxis2': {'anchor': 'y', 'overlaying': 'x', 'side': 'top'},
        'yaxis': {'side': 'left', 'title': {'text': 'Frequency'}},
        'yaxis2': {'overlaying': 'y', 'side': 'right', 'title': {'text': 'KDE'}}
    }))

    all_figures.append(figure)

# Save all figures as HTML files
for i, figure in enumerate(all_figures):
    plot(figure, filename=f'histogram_{numeric_columns[i]}.html')


In [108]:
import pandas as pd
import plotly.express as px
import plotly.io as pio

# Set the default renderer to 'browser'
pio.renderers.default = 'browser'

# Assuming 'df' is already defined and contains the necessary data

# Handle special index types
if isinstance(df, (pd.DatetimeIndex, pd.MultiIndex)):
    df = df.to_frame(index=False)

# Reset the index and ensure column names are strings
df = df.reset_index().drop('index', axis=1, errors='ignore')
df.columns = [str(c) for c in df.columns]  # Update columns to strings in case they are numbers

# Get the list of numeric columns
numeric_columns = df.select_dtypes(include='number').columns.tolist()

# Initialize a dictionary to store statistics for each column
all_stats = {}

# Iterate through each numeric column and calculate statistics
for column in numeric_columns:
    # Main statistics
    stats = df[column].describe().to_frame().T

    # Additional statistical measures
    stats['sum'] = df[column].sum()
    stats['median'] = df[column].median()
    stats['var'] = df[column].var()
    stats['sem'] = df[column].sem()

    # Outliers
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    outliers = df[(df[column] < (Q1 - 1.5 * IQR)) | (df[column] > (Q3 + 1.5 * IQR))][column]
    unique_outliers = outliers.unique()
    # Unique values
    unique_values = df[column].nunique()

    # Store results in the dictionary
    all_stats[column] = {
        'main_stats': stats,
        'outliers': unique_outliers,
        'unique_values': unique_values
    }

    # Display the statistics for the current column
    print(f"Statistics for column: {column}")
    print("Main Statistics:")
    print(stats)
    print("\nOutliers:")
    print(unique_outliers)
    print("\nUnique Values:")
    print(unique_values)
    print("-" * 50)
    
    # Create a box plot for the current column
    fig = px.box(df, y=column, title=f'Box Plot for {column}')
    #fig.show()

    



Statistics for column: S1_Temp
Main Statistics:
           count       mean       std    min    25%    50%    75%    max  \
S1_Temp  10129.0  25.454012  0.351351  24.94  25.19  25.38  25.63  26.38   

               sum  median       var       sem  
S1_Temp  257823.69   25.38  0.123447  0.003491  

Outliers:
[26.31 26.38]

Unique Values:
24
--------------------------------------------------
Statistics for column: S2_Temp
Main Statistics:
           count       mean       std    min    25%    50%    75%   max  \
S2_Temp  10129.0  25.546059  0.586325  24.75  25.19  25.38  25.63  29.0   

               sum  median       var       sem  
S2_Temp  258756.03   25.38  0.343778  0.005826  

Outliers:
[26.38 26.81 27.   26.94 27.19 27.25 27.44 27.5  27.56 27.75 27.94 27.69
 27.38 27.31 27.63 27.81 28.   28.19 28.13 28.06 27.88 28.31 28.38 28.5
 28.44 28.25 28.56 28.63 28.69 28.81 28.75 28.88 28.94 29.   27.13 27.06
 26.88 26.75 26.69 26.63 26.56 26.5  26.44 26.31]

Unique Values:
69
-----------